In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
import time

data = pd.read_parquet('train_data.parquet', engine='pyarrow')

In [2]:

# Полезные частые функции

def metrics(y_true, y_preds):
    acc = sklearn.metrics.accuracy_score(y_true, y_preds)  
    rec = sklearn.metrics.recall_score(y_true, y_preds, average=None)
    f1 = sklearn.metrics.f1_score(y_true, y_preds)
    prec = 	sklearn.metrics.precision_score(y_true, y_preds)
    auc = sklearn.metrics.roc_auc_score(y_true, y_preds)
    print('test-accuracy:', acc, ' \ntest-recall:', rec, '\ntest-f1_score:', f1, '\ntest-precision_score:', prec, '\ntest-roc_auc_score:', auc)
    


def equallySample(data):
    y_data = data["target"]
    x_data = data.drop(useless_columns, axis=1)
    sample_size = 17000 # Задайте желаемое количество строк для каждого класса
    frames = []
    classes = data['target'].unique()

    for i in classes:
        if i == 0:  
            g = data[data['target'] == i].sample(sample_size)
        else:
            g = data[data['target'] == i].sample(sample_size)
        frames.append(g)

    equally_sampled = pd.concat(frames)
    equally_sampled['target'].value_counts()

    X_train_equally_sampled, X_test_equally_sampled, y_train_equally_sampled,  y_test_equally_sampled = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)


In [ ]:

useless_columns = ['id', 'target', 'sample_ml_new', 'feature68', 'feature69', 'feature144', 'feature160',
                   'feature292', 'feature406', 'feature407', 'feature496', 'feature511',
                   'feature625', 'feature661', 'feature663', 'feature669', 'feature678',
                   'feature683', 'feature686', 'feature710', 'feature756', 'feature761',
                   'feature765', 'feature801', 'feature802', 'feature806', 'feature807',
                   'feature808', 'feature809', 'feature816', 'feature818', 'feature819',
                   'feature955', 'feature956', 'feature957', 'feature958', 'feature959',
                   'feature960', 'feature961', 'feature962', 'feature963', 'feature964',
                   'feature965', 'feature966', 'feature967', 'feature968', 'feature969',
                   'feature970', 'feature971', 'feature972', 'feature973', 'feature974', 
                   'feature975', 'feature976', 'feature977', 'feature978', 'feature979',
                   'feature980', 'feature981', 'feature982', 'feature983', 'feature984', 
                   'feature1005', 'feature1006', 'feature1007', 'feature1008', 'feature1009',
                   'feature1010', 'feature1011', 'feature1012', 'feature1013', 'feature1014',
                   'feature1015', 'feature1016', 'feature1017', 'feature1018', 'feature1019',
                   'feature1020', 'feature1021', 'feature1022', 'feature1023', 'feature1024',
                   'feature1025', 'feature1026', 'feature1027', 'feature1028', 'feature1029',
                   'feature1030', 'feature1031', 'feature1032', 'feature1033', 'feature1034']


y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)
x_data

In [ ]:
X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)


In [ ]:
# EQual size of targets

print(y_train.value_counts())
print(y_test.value_counts())


sample_size = 17000 # Задайте желаемое количество строк для каждого класса
frames = []
classes = data['target'].unique()

for i in classes:
    if i == 0:  
        g = data[data['target'] == i].sample(sample_size)
    else:
        g = data[data['target'] == i].sample(sample_size)
    frames.append(g)

equally_sampled = pd.concat(frames)
equally_sampled['target'].value_counts()

X_train_equally_sampled, X_test_equally_sampled, y_train_equally_sampled,  y_test_equally_sampled = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)
"""X_train_xgb_equally_sampled = xgb.DMatrix(X_train_equally_sampled, label=y_train_equally_sampled)
X_test_xgb_equally_sampled = xgb.DMatrix(X_test_equally_sampled, label=y_test_equally_sampled)"""

In [ ]:
equally_sampled['target'].value_counts()

In [ ]:
xgb_data = xgb.DMatrix(x_data, label=y_data)

In [ ]:
params = {'max_depth': 4,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'map']}
eval_list = [(X_test_xgb, "Everything")]
num_rounds = 22

In [ ]:
evals_result = dict()

def objective(n_estimators, max_depth, classes_weight_o, learning_rate, gamma):
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    classes_weight_o = int(classes_weight_o)
    classes_weights = class_weight.compute_sample_weight(
    class_weight={0:1, 1:classes_weight_o},
    y=y_train
)   

    
    xgb_model = xgb.XGBClassifier(  n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    learning_rate=learning_rate,
                                    booster="gbtree",
                                    n_jobs=-1,
                                    eval_metric=['auc', 'pre'],
                                    gamma=gamma)

    xgb_model.fit(X_train, y_train, eval_set=eval_list, sample_weight=classes_weights)
    preds = xgb_model.predict(X_test)
    return sklearn.metrics.f1_score(y_test, preds)

"""parameters = {
    'max_depth': [4, 20],
    'learning_rate': [0.2],
    'n_estimators': [20, 40],
}

params = {'max_depth': 6,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'pre'],
          'class_weight': {0:0.1, 1:100}}"""
eval_list = [(X_test, y_test)]

from sklearn.utils import class_weight
"""classes_weights = class_weight.compute_sample_weight(
    class_weight={0:1, 1:22},
    y=y_train
)"""
#clf = sklearn.model_selection.GridSearchCV(xgb_model, parameters, scoring='f1_micro', n_jobs=-1, verbose=10)

search_space = {
    'max_depth': (4, 20),
    'n_estimators': (10, 70),
    "classes_weight_o": (10, 40),
    'learning_rate': (0.001, 0.2),
    'gamma': (0.01, 5)
}
import bayes_opt
optimizer = bayes_opt.BayesianOptimization(
                                f=objective,
                                pbounds=search_space,
                                random_state=12,
                                allow_duplicate_points=True
                              ) 

opt = optimizer.maximize(init_points=2, n_iter=20)

In [ ]:
optimizer.max['params']

In [ ]:
classes_weights = class_weight.compute_sample_weight(
    class_weight={0:1, 1:31},
    y=y_train
)   

xgb_model = xgb.XGBClassifier(  gamma=4.8,
                                learning_rate=0.152,
                                max_depth=8,
                                n_estimators=18,
                                booster="gbtree",
                                n_jobs=-1,
                                eval_metric=['auc', 'pre'],
                                early_stopping_rounds=10)

xgb_model.fit(X_train, y_train, eval_set=eval_list, sample_weight=classes_weights)
preds = xgb_model.predict(X_test)
metrics(y_test, preds)


"""{'classes_weight_o': 31.438781865708886,
 'gamma': 4.806812881469824,
 'learning_rate': 0.15290427033360873,
 'max_depth': 8.582152510429442,
 'n_estimators': 18.513852056962605}
                                    test-accuracy: 0.7220024017736174  
                                    test-recall: [0.72583707 0.61577552] 
                                    test-f1_score: 0.13371074915441264 
                                    test-precision_score: 0.07499798175506579 
                                    test-roc_auc_score: 0.6708062939632549"""

In [ ]:

y_preds = xgb_model.predict(X_test)
y_true = list(y_test)
print(*y_preds)

#ev = clf.eval(X_test)
metrics(y_true, y_preds)
#print(ev, ' eval-recall:')

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_preds, y_test))
cm = confusion_matrix(y_preds, y_test)
acc = cm.diagonal().sum()/cm.sum()
print(acc)

# GRIDSEARCH XGBOOST

In [ ]:
params = {
    'max_depth': range(2,10,2),
    'learning_rate': np.linspace(.1, .6, 6),
    'min_child_weight': range(1,10,2),
}

grid = sklearn.model_selection.GridSearchCV(
    estimator = xgb.XGBClassifier(n_jobs=-1,
                              n_estimators=500,
                              random_state=0),
    param_grid = params,
    scoring='f1',  # <------Use `scoring` instead of `eval_metric`
)

eval_set = [(X_train_equally_sampled, y_train_equally_sampled),
            (X_test_equally_sampled, y_test_equally_sampled)]

grid.fit(X_train_equally_sampled, y_train_equally_sampled,
         eval_set=eval_set,
         early_stopping_rounds=25,
)

In [ ]:
data

In [ ]:
data['feature5'].describe()

In [ ]:
data['feature2'].value_counts()

In [ ]:
x=140
corr = data[['target', 'feature'+str(x), 'feature'+str(x+1), 'feature'+str(x+2), 'feature'+str(x+3), 
                'feature'+str(x+4), 'feature'+str(x+5), 'feature'+str(x+6), 'feature'+str(x+7), 'feature'+str(x+8),
                'feature'+str(x+9)]].corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
boxplot = data.boxplot("feature5")

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import xgboost as xgb

useless_columns = ['id', 'target', 'sample_ml_new', 'feature68', 'feature69', 'feature144', 'feature160',
                   'feature292', 'feature406', 'feature407', 'feature496', 'feature511',
                   'feature625', 'feature661', 'feature663', 'feature669', 'feature678',
                   'feature683', 'feature686', 'feature710', 'feature756', 'feature761',
                   'feature765', 'feature801', 'feature802', 'feature806', 'feature807',
                   'feature808', 'feature809', 'feature816', 'feature818', 'feature819',
                   'feature955', 'feature956', 'feature957', 'feature958', 'feature959',
                   'feature960', 'feature961', 'feature962', 'feature963', 'feature964',
                   'feature965', 'feature966', 'feature967', 'feature968', 'feature969',
                   'feature970', 'feature971', 'feature972', 'feature973', 'feature974', 
                   'feature975', 'feature976', 'feature977', 'feature978', 'feature979',
                   'feature980', 'feature981', 'feature982', 'feature983', 'feature984', 
                   'feature1005', 'feature1006', 'feature1007', 'feature1008', 'feature1009',
                   'feature1010', 'feature1011', 'feature1012', 'feature1013', 'feature1014',
                   'feature1015', 'feature1016', 'feature1017', 'feature1018', 'feature1019',
                   'feature1020', 'feature1021', 'feature1022', 'feature1023', 'feature1024',
                   'feature1025', 'feature1026', 'feature1027', 'feature1028', 'feature1029',
                   'feature1030', 'feature1031', 'feature1032', 'feature1033', 'feature1034']

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]

x_data = data.drop(useless_columns, axis=1)
print(x_data.columns)

In [ ]:
X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

X_train_xgb = xgb.DMatrix(X_train, label=y_train)
X_test_xgb = xgb.DMatrix(X_test, label=y_test)

In [ ]:
from sklearn.impute import SimpleImputer

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns


cols_with_missing = [col for col in train_num_cleaned.columns
                                if test_num_cleaned[col].isnull().any()]
print(cols_with_missing)
for col in cols_with_missing:
    print(col, test_num_cleaned[col].isnull().any())

In [ ]:
# Изменение веса классов

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_data), y=y_data)

# Создание модели с взвешиванием классов
model = RandomForestClassifier(n_estimators=30, class_weight={0: 0.5, 1: 500.0})
model.fit(train_num_cleaned, y_train)

In [ ]:
preds = model.predict(test_num_cleaned)
preds = list(map(round, preds))
y_true = list(y_test)
print(preds.count(1))
#ev = bst.eval(X_test_xgb)


rec = sklearn.metrics.recall_score(y_true, preds)
f1 = sklearn.metrics.f1_score(y_true, preds)
prec = 	sklearn.metrics.precision_score(y_true, preds)
auc = sklearn.metrics.roc_auc_score(y_true, preds)
print(' test-recall:', rec, ' test-f1_score:', f1, ' test-precision_score:', prec, ' test-roc_auc_score:', auc)

In [ ]:
# Методы борьбы с дисбалансом данных
import sklearn
print(sklearn.__version__)
import pandas as pd
from imblearn.over_sampling import SMOTE

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

# Создание экземпляра SMOTE
smote = SMOTE()

# Применение SMOTE к данным
X_resampled, y_resampled = smote.fit_resample(train_num_cleaned, y_train)

X_resampled_xgb = xgb.DMatrix(X_resampled, y_resampled)
X_test_xgb = xgb.DMatrix(test_num_cleaned, y_test)


In [ ]:
params = {'max_depth': 8,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'pre']}
eval_list = [(X_test_xgb, "Everything")]
num_rounds = 80

In [ ]:
y_resampled.value_counts()

In [ ]:
evals_result = dict()
bst = xgb.train(params, X_resampled_xgb, num_rounds, eval_list, evals=X_test_xgb, evals_result=evals_result)
y_preds = bst.predict(X_test_xgb)
preds = list(map(round, preds))
y_true = list(y_test)
metrics(y_true, preds)

In [ ]:
metrics(y_true, preds)

In [ ]:
# Обучение каждой моделей члена на подмножестве данных

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

X_train_xgb = xgb.DMatrix(train_num_cleaned, y_train)
X_test_xgb = xgb.DMatrix(test_num_cleaned, y_train)

eval_list = [(X_test_xgb, "Everything")]
num_rounds = 40
ensemble = [xgb.train({'max_depth': x,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'pre']}, X_train_xgb, num_rounds, eval_list, evals=X_test_xgb, evals_result=evals_result) for x in range(2, 10)]
ind = 1

"""for model in ensemble:
    sample_indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_subset, y_subset = X_train[sample_indices], y_train[sample_indices]
    model.fit(X_subset, y_subset)
    print(f"Модель {ind} обучена")
    ind +=1"""

In [ ]:
from statistics import mode
predictions = np.array([model.predict(X_test_xgb) for model in ensemble])
new_predictions=[]
for x in range(len(predictions[0])):
    new_predictions.append(np.array([predictions[y][x] for y in range(len(predictions))]))
ensemble_predictions = [int(mode(x.round())) for x in new_predictions]
# Используем np.argmax для нахождения индекса наиболее часто встречающегося значения

metrics(y_test, ensemble_predictions)

In [ ]:
# Кросс-обучение

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

X_train_xgb = xgb.DMatrix(train_num_cleaned, y_train)
X_test_xgb = xgb.DMatrix(test_num_cleaned, y_train)

In [ ]:
data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns



In [3]:
# Fucking useless parts of code
data = pd.read_parquet('train_data.parquet', engine='pyarrow')

columns_with_few_unique_values = ['feature642', 'feature756', 'sample_ml_new', 'id']

drop_colums = {'feature642', 'feature756', 'sample_ml_new', 'id'}
for col in data.iloc[:, 3:]:
    if np.unique(data[col]).shape == (1,):
        drop_colums.add(col)
correl = data.drop(['sample_ml_new', 'id'], axis=1).corr()
for i in correl.iloc[1:, 1][np.asarray(np.abs(correl.iloc[1:, 1]) < 0.0003)].index:
    drop_colums.add(col)
    columns_with_few_unique_values.append(col)
#print(*columns_with_few_unique_values, sep="\n")
drop_colums

In [6]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import sklearn
import sklearn.model_selection
from imblearn.over_sampling import ADASYN

data = pd.read_parquet('train_data.parquet', engine='pyarrow')

useless_columns = ['id', 'sample_ml_new', 'feature68', 'feature69', 'feature144', 'feature160',
                   'feature292', 'feature406', 'feature407', 'feature496', 'feature511',
                   'feature625', 'feature661', 'feature663', 'feature669', 'feature678',
                   'feature683', 'feature686', 'feature710', 'feature756', 'feature761',
                   'feature765', 'feature801', 'feature802', 'feature806', 'feature807',
                   'feature808', 'feature809', 'feature816', 'feature818', 'feature819',
                   'feature955', 'feature956', 'feature957', 'feature958', 'feature959',
                   'feature960', 'feature961', 'feature962', 'feature963', 'feature964',
                   'feature965', 'feature966', 'feature967', 'feature968', 'feature969',
                   'feature970', 'feature971', 'feature972', 'feature973', 'feature974', 
                   'feature975', 'feature976', 'feature977', 'feature978', 'feature979',
                   'feature980', 'feature981', 'feature982', 'feature983', 'feature984', 
                   'feature1005', 'feature1006', 'feature1007', 'feature1008', 'feature1009',
                   'feature1010', 'feature1011', 'feature1012', 'feature1013', 'feature1014',
                   'feature1015', 'feature1016', 'feature1017', 'feature1018', 'feature1019',
                   'feature1020', 'feature1021', 'feature1022', 'feature1023', 'feature1024',
                   'feature1025', 'feature1026', 'feature1027', 'feature1028', 'feature1029',
                   'feature1030', 'feature1031', 'feature1032', 'feature1033', 'feature1034']
#useless_columns = ['id', *columns_with_few_unique_values[1:]]

y_data = data["target"]
x_data = data.drop([*useless_columns, 'target'], axis=1)

sample_size = 17000 # Задайте желаемое количество строк для каждого класса
frames = []
classes = data['target'].unique()

for i in classes:
    if i == 0:  
        g = data[data['target'] == i].sample(sample_size)
    else:
        g = data[data['target'] == i].sample(sample_size)
    frames.append(g)

equally_sampled = pd.concat(frames)
equally_sampled = equally_sampled.drop(useless_columns, axis=1)
y_equally_sampled = pd.DataFrame(equally_sampled['target'])
x_equally_sampled = pd.DataFrame(equally_sampled.drop(['target'], axis=1))

X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_equally_sampled, y_equally_sampled, test_size=0.33, shuffle=True)

#X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler

# ЗАПОЛНЕНИЕ ПУСТОТ TRAIN и TEST
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns
print(y_train.value_counts())

from sklearn.ensemble import IsolationForest

# УДАЛЕНИЕ ВЫБРОСОВ
clf = IsolationForest(contamination=0.05)
clf.fit(train_num_cleaned)
y_pred_train = clf.predict(train_num_cleaned)
y_pred_test = clf.predict(test_num_cleaned)

train_num_cleaned = train_num_cleaned[y_pred_train == 1]
y_train = y_train[y_pred_train == 1]
test_num_cleaned = test_num_cleaned[y_pred_test == 1]
y_test = y_test[y_pred_test == 1]

# НОРМАЛИЗАЦИЯ
scaler = MinMaxScaler()
scaler.fit(train_num_cleaned)
data_train = scaler.transform(train_num_cleaned)
data_test = scaler.transform(test_num_cleaned)


"""# ВЫРАВНИВАНИЕ ДИСБАЛАНСА КЛАССОВ
ada = ADASYN(sampling_strategy='auto', random_state=0, n_neighbors=5)
data_train, y_train = ada.fit_resample(data_train, y_train)
print(y_train.value_counts())
"""

# ПОДГОТОВКА ВСЕГО ДАТАСЕТА
all_data = x_data.select_dtypes(exclude=['object'])
all_data_cleaned = imputer.transform(all_data)
y_pred = clf.predict(all_data_cleaned)
all_data_cleaned = all_data_cleaned[y_pred == 1]
y_data = y_data[y_pred==1]
all_data_cleaned = pd.DataFrame(all_data_cleaned)
all_data_cleaned.columns = x_data.columns
all_data = scaler.transform(all_data_cleaned)



target
0         11457
1         11323
Name: count, dtype: int64


C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [7]:
data_train

array([[0.78875639, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.70300965, 1.        , 0.14184397, ..., 0.53973893, 0.54103455,
        0.49632237],
       ...,
       [0.97444634, 0.97555429, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.9290176 , 0.04604889, 0.88652482, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 0.14893617, ..., 0.        , 0.        ,
        0.        ]])

In [8]:
print(data_train.shape)
print(y_train.shape)

(21641, 987)
(21641, 1)


In [206]:

model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(816, ), activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(1, activation = 'sigmoid')
])



class_weights = {0: 0.5,
                 1: 1.1}
model.compile(optimizer="sgd",
              loss='binary_crossentropy',
              metrics=['AUC'])

model.summary()



C:\Users\kseni\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_49"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_159 (Dense)                    │ (None, 128)                 │         104,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_62 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_160 (Dense)                    │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_63 (Dropout)                 │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_161 (Dense)                    │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_162 (Dense)                    │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 109,249 (426.75 KB)

 Trainable params: 109,249 (426.75 KB)

 Non-trainable params: 0 (0.00 B)

In [207]:
model.fit(x=data_train, y=y_train, batch_size=128, epochs=15, verbose=1, validation_data=(data_test, y_test))

Epoch 1/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - AUC: 0.5562 - loss: 0.6883 - val_AUC: 0.6530 - val_loss: 0.6839
Epoch 2/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6322 - loss: 0.6652 - val_AUC: 0.6705 - val_loss: 0.6473
Epoch 3/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6503 - loss: 0.6578 - val_AUC: 0.6748 - val_loss: 0.6990
Epoch 4/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6611 - loss: 0.6526 - val_AUC: 0.6819 - val_loss: 0.6546
Epoch 5/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6679 - loss: 0.6486 - val_AUC: 0.6838 - val_loss: 0.6613
Epoch 6/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6812 - loss: 0.6416 - val_AUC: 0.6854 - val_loss: 0.7046
Epoch 7/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6854 - loss: 0.6390 - val_AUC: 0.6874 - val_loss: 0.6486
Epoch 8/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6946 - loss: 0.6331 - val_AUC: 0.6894 - val_loss: 0.6873
Epoch 9/15
382/382 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AU

In [208]:
model.evaluate(all_data, y_data)

15341/15341 ━━━━━━━━━━━━━━━━━━━━ 10s 624us/step - AUC: 0.7036 - loss: 0.6108


[0.6122022867202759, 0.7066034078598022]

In [210]:
proba = model.predict(all_data)
threshold = 0.5 # Установите свой порог
predictions = (proba > threshold).astype('int32')
print("thr = ", threshold)
metrics(y_data, predictions)
threshold = 0.4 # Установите свой порог
predictions = (proba > threshold).astype('int32')
print("thr = ", threshold)
metrics(y_data, predictions)
threshold = 0.6 # Установите свой порог
predictions = (proba > threshold).astype('int32')
print("thr = ", threshold)
metrics(y_data, predictions)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         104,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 104,707 (409.02 KB)

 Trainable params: 104,705 (409.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

15341/15341 ━━━━━━━━━━━━━━━━━━━━ 9s 572us/step
thr =  0.5
test-accuracy: 0.6749534530313586  
test-recall: [0.67548783 0.66052587] 
test-f1_score: 0.12675670942603215 
test-precision_score: 0.07010502739187022 
test-roc_auc_score: 0.6680068492349086
thr =  0.4
test-accuracy: 0.5046485856647559  
test-recall: [0.49355577 0.80414076] 
test-f1_score: 0.10391093979739614 
test-precision_score: 0.05554417453926582 
test-roc_auc_score: 0.6488482652045351
thr =  0.6
test-accuracy: 0.802840485473679  
test-recall: [0.8140119  0.50122626] 
test-f1_score: 0.15368741365138772 
test-precision_score: 0.0907579340899937 
test-roc_auc_score: 0.6576190804872106


In [ ]:
model.save("model_ver_7330.h5")

In [ ]:
model_name = "model_ver_7330.h5"

model_to_subm = tf.keras.models.load_model(model_name)
model_to_subm.summary()

In [ ]:
# CODE TO LOAD DATA AND SUBMISSION

model_to_subm = tf.keras.models.load_model(model_name)
#model_to_subm.evaluate(all_data, y_data)

test_dataset = pd.read_parquet('test_sber.parquet', engine='pyarrow')
useless_columns = ['id', *columns_with_few_unique_values[1:]]

test_dataset = test_dataset.drop(useless_columns, axis=1)
test_dataset_cleaned = imputer.transform(test_dataset)
test_dataset_cleaned = pd.DataFrame(test_dataset_cleaned)
test_dataset_cleaned.columns = test_dataset.columns
test_dataset = scaler.transform(test_dataset_cleaned)

pred = model_to_subm.predict(test_dataset)
pred_binary = np.round(pred).astype(int)

submission = pd.read_csv("sample_submission.csv")
submission["target_prob"] = pred
submission["target_bin"] = pred_binary
submission.to_csv("submission.csv", index=False)

In [214]:
model = sklearn.linear_model.LogisticRegression(max_iter=1000, 
                                                verbose=2, 
                                                n_jobs=-1)

model.fit(data_train, y_train)

C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   42.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   42.7

LogisticRegression(max_iter=1000, verbose=2)

In [216]:
metrics(model.predict(data_test), y_test)

test-accuracy: 0.6654883940143749  
test-recall: [0.79762623 0.47866894] 
test-f1_score: 0.5424657534246575 
test-precision_score: 0.6258832279657865 
test-roc_auc_score: 0.6381475870629858


In [219]:
model = sklearn.neighbors.KNeighborsClassifier(n_neighbors=3)
model.fit(data_train, y_train)

C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [220]:
metrics(model.predict(data_test), y_test)

test-accuracy: 0.38824083893012845  
test-recall: [0.74105012 0.3269018 ] 
test-f1_score: 0.47656013711059586 
test-precision_score: 0.8789512830048345 
test-roc_auc_score: 0.533975958697683


In [225]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier


In [221]:

model = RandomForestClassifier()
model.fit(data_train, y_train)

C:\Users\kseni\AppData\Local\Temp\ipykernel_17960\1731829732.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(data_train, y_train)


RandomForestClassifier()

In [223]:
metrics(model.predict(data_test), y_test)

test-accuracy: 0.7248733356898787  
test-recall: [0.75493227 0.60442478] 
test-f1_score: 0.46738138686131386 
test-precision_score: 0.3809966530308665 
test-roc_auc_score: 0.679678526011862


In [226]:
model = AdaBoostClassifier()
model.fit(data_train, y_train)

C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier()

In [227]:
metrics(model.predict(data_test), y_test)

test-accuracy: 0.6931188877106162  
test-recall: [0.75669158 0.51864108] 
test-f1_score: 0.474422358995056 
test-precision_score: 0.43715135738192634 
test-roc_auc_score: 0.6376663304137686


In [245]:
%pip install  optuna

  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/4c/6a/219a431aaf81b3eb3070fd2d58116baa366d3072f43bbcc87dc3495b7546/optuna-3.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/7f/50/9fb3a5c80df6eb6516693270621676980acd6d5a9a7efdbfa273f8d616c7/alembic-1.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/f3/18/3e867ab37a24fdf073c1617b9c7830e06ec270b1ea4694a624038fc40a03/colorlog-6.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for Mako from https://files.pythonhosted.org/packages/2b/8d/9f11d0b9ac521febb806e7f30dc5982d0f4f5821217712c59005fbc5c1e3/Mako-1.3.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
   -- ------------------------------------ 30.7/413.4 kB 660.6 kB/s eta 0:00:01
   --- ----------------------------------- 41.0/413

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import optuna

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000)
    criterion = trial.suggest_categorical("criterion", ['friedman_mse'])
    max_depth = trial.suggest_int("max_depth", 1, 11)
    min_samples_split = trial.suggest_float("min_samples_split", 0.01, 1)
    max_features = trial.suggest_categorical("features", ['sqrt', 'log2'])
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.1)
    
    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        max_features=max_features,
        learning_rate=learning_rate
    )
    
    score = cross_val_score(model, data_train, y_train, cv=3, scoring='roc_auc')
    accuracy = score.mean()
    return accuracy


study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.RandomSampler()
)

study.optimize(objective, n_trials=100)

[I 2024-03-13 16:16:09,747] A new study created in memory with name: no-name-cf98d91a-d262-4bff-bcbb-f18afad2eebc
C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-13 16:16:36,674] Trial 0 finished with value: 0.822375440008067 and parameters: {'n_estimato

KeyboardInterrupt: 

In [13]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
best_param = {
    'n_estimators': 1000,
    'criterion': 'friedman_mse',
    'max_depth': 9,
    'min_samples_split': 0.071028349581022217,
    'learning_rate': 0.072415345054296616
}

model = GradientBoostingClassifier(**best_param, verbose=3, )
model.fit(data_train, y_train)

C:\Users\kseni\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           1.3650           55.90m
         2           1.3463           56.01m
         3           1.3299           55.72m
         4           1.3153           55.05m
         5           1.3012           55.69m
         6           1.2890           55.65m
         7           1.2778           56.27m
         8           1.2683           55.82m
         9           1.2592           55.69m
        10           1.2509           55.81m
        11           1.2436           55.76m
        12           1.2365           55.84m
        13           1.2291           55.90m
        14           1.2232           55.91m
        15           1.2171           55.89m
        16           1.2117           55.66m
        17           1.2069           55.75m
        18           1.2018           55.88m
        19           1.1972           55.87m
        20           1.1923           56.05m
        21           1.1882           56.11m
        2

GradientBoostingClassifier(learning_rate=0.07241534505429661, max_depth=9,
                           min_samples_split=0.07102834958102222,
                           n_estimators=1000, verbose=3)

In [11]:
import pickle
pickle.dump(model ,open("model_ver_3.dat",'wb'))

In [14]:
metrics(model.predict(data_test), y_test)

test-accuracy: 0.6719044034625518  
test-recall: [0.66468953 0.67934263] 
test-f1_score: 0.6709446069642351 
test-precision_score: 0.662751677852349 
test-roc_auc_score: 0.6720160803144373


In [16]:
test_dataset = pd.read_parquet('test_sber.parquet', engine='pyarrow')
useless_columns = ['id', *cols_with_missing]

test_dataset = test_dataset.drop(useless_columns, axis=1)
test_dataset_cleaned = imputer.transform(test_dataset)
test_dataset_cleaned = pd.DataFrame(test_dataset_cleaned)
test_dataset_cleaned.columns = test_dataset.columns
test_dataset = scaler.transform(test_dataset_cleaned)

pred = model.predict_proba(test_dataset)
print(pred)
pred_binary = pred.argmax(axis=1)
print(pred_binary)
pred = pred.max(axis=1)
print(pred)
pred = [pred[x] if pred_binary[x]==1 else 1-pred[x] for x in range(len(pred_binary))]

submission = pd.read_csv("sample_submission.csv")
submission["target_prob"] = pred
submission["target_bin"] = pred_binary
submission.to_csv("submission.csv", index=False)

NameError: name 'cols_with_missing' is not defined

In [ ]:
from imblearn.pipeline import imbPipeline
pipeline = imbPipeline([
    
])